In [11]:
import  requests 
import pandas as pd

from urllib.error import URLError, HTTPError
from urllib3.exceptions import SSLError

from requests.exceptions import HTTPError
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from socket import timeout

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options 
import time
import sys


import random

In [14]:
class collect_jobs_data():
    
    def __init__(self,keywrods, driver_path,page_scroll_rate):
        
        self.page_scroll_rate = page_scroll_rate
        self.keywords = keywrods
        self.driver_path = driver_path
        
        self.full_url = self.process_keyword()
        self.driver = self.iniate_selenium_driver()
        self.number_of_jobs = self.extract_no_jobs()
        self.links_titles = self.extract_job_links_titles()

    def process_keyword(self):
        
        keywords = self.keywords
        keywords2 = keywords
        keywords = '%20'.join(keywords.split())
        keywords2 = '_'.join(keywords2.split())

        url = 'https://www.linkedin.com/jobs/search'
        t1 = f'?keywords={keywords}&location=&geoId=&'
        t2 = 'trk=homepage-jobseeker_jobs-search-bar_search-submit&position=1&pageNum=0'

        full_url = f'{url}{t1}{t2}'
        print(f'current Keyword: {keywords2}')

        return full_url


    def iniate_selenium_driver(self):
        
        option = webdriver.ChromeOptions()
        option.add_argument('headless')
        driver = webdriver.Chrome(executable_path= self.driver_path,options=option)

        driver.set_page_load_timeout('30')
        driver.get(self.full_url)
        

        return driver


    def extract_no_jobs(self):
        
        n_jobs = self.driver.find_element_by_css_selector('h1>span').get_attribute('innerText')
        n_jobs = n_jobs.strip(',')
        
        remove_lst = [',', '+', '.']
        for i in remove_lst:
            n_jobs = n_jobs.replace(i, '')

        n_jobs = int(n_jobs)
        print('number of jobs found : ', n_jobs)
        return n_jobs


    def extract_job_links_titles(self):
        #     job_id= []
        #     company_name = []
        #     location = []
        #     date = []

        job_title = []
        job_link = []
        i = 2

        # while i <= int(self.number_of_jobs/25)+1:
        if self.number_of_jobs < self.page_scroll_rate:
            self.page_scroll_rate = self.number_of_jobs
            
        while i <= int(self.page_scroll_rate / 25) + 1:

            self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            i = i + 1
            try:
                self.driver.find_element_by_xpath('/html/body/main/div/section/button').click()
                time.sleep(5)
            except:
                pass
                time.sleep(5)

        job_lists = self.driver.find_element_by_class_name('jobs-search__results-list')
        jobs = job_lists.find_elements_by_tag_name('li')
        len(jobs)

#         jobs = jobs[:2]

        for job in jobs:
            job_title0 = job.find_element_by_css_selector('h3').get_attribute('innerText')
            job_title.append(job_title0)

            job_link0 = job.find_element_by_css_selector('a').get_attribute('href')
            job_link.append(job_link0)

        self.driver.close()

        return [job_title, job_link]


    def extract_jobs_postings(self):
        
        job_title,job_link = self.links_titles

        jobs_details = []

        for jl in job_link:

            option = webdriver.ChromeOptions()
            option.add_argument('headless')
            driver = webdriver.Chrome(executable_path= self.driver_path,options=option)
            driver.set_page_load_timeout('15')

            try:
                driver.get(jl)

                job_detail = driver.find_element_by_class_name('core-section-container__content').get_attribute('innerText')
                job_title = driver.find_element_by_css_selector('h1').get_attribute('innerText')

                jobs_details.append([job_title, job_detail])

                driver.close()

            except:
#                 print(f'error in: {jl}')
                
                driver.close()
                pass

        return jobs_details


In [15]:
# keywords = ['Advertising & Public Relations',
#  'Aerospace Engineering',
#  'Agriculture',
#  'Animal Sciences',
#  'Anthropology & Archeology',

keywords = [
 'Applied Mathematics',
 'Architecture',
 'Art & Music Education',
 'History',
 'Biological Engineering',
 'Biomedical Engineering',
 'Business Management & Administration',
 'Business Economics',
 'Chemical Engineering',
 'Civil Engineering']



for keyword in keywords: 
    obj = collect_jobs_data(keyword,'selenium_p/chromedriver',500)   # keyword, driver path, no of scrolls in the page
    time.sleep(5)
    job_postings = obj.extract_jobs_postings() #[[all titles], [all descriptions]]
    time.sleep(5)
    print('jobs collected: ',len(job_postings))
    df = pd.DataFrame(job_postings, columns =['job-title','description'])
    df.to_csv(f'{keyword}_{len(job_postings)}_records.csv', encoding='utf-8', index=False)
print('Done')

current Keyword: Applied_Mathematics
number of jobs found :  8000
jobs collected:  166
current Keyword: Architecture
number of jobs found :  325000
jobs collected:  168
current Keyword: Art_&_Music_Education
number of jobs found :  247000
jobs collected:  171
current Keyword: History
number of jobs found :  345000
jobs collected:  165
current Keyword: Biological_Engineering
number of jobs found :  2000
jobs collected:  164
current Keyword: Biomedical_Engineering
number of jobs found :  7000
jobs collected:  161
current Keyword: Business_Management_&_Administration
number of jobs found :  749000
jobs collected:  156
current Keyword: Business_Economics
number of jobs found :  39000
jobs collected:  155
current Keyword: Chemical_Engineering
number of jobs found :  28000
jobs collected:  156
current Keyword: Civil_Engineering
number of jobs found :  56000
jobs collected:  142
Done


In [42]:
df.head()

,job-title,description
0,SFA - Books Marketing,Job Summary\n\nDESCRIPTION\n\nThe Books team (...
1,"Sr Manager, Analytics, Insights & Reporting",Job DescriptionPosition Summary\n\nNordstrom C...
2,Data Scientist,"As a Data Scientist, you will collaborate with..."
3,After School Teacher,Actual pay may be different — this range is es...
4,Program Instructor-Mathematics,"Location: Sioux Falls, South Dakota Campus\n\n..."


In [43]:
df['job-title'].iloc[0]

'SFA - Books Marketing'

In [45]:
print(df['description'].iloc[0])

Job Summary

DESCRIPTION

The Books team (Books + Kindle Content) is looking for a results-oriented and data-driven professional to lead the finance function for the Books marketing team. This position will work closely alongside a team of world-class marketers through new development initiatives, marketing opportunities, seasonal events strategy and inventing ways to meet new customers and deliver demand acceleration through advertising strategies and onsite CX improvements.

The role requires an individual with a passion for books, the ability to dive deep into the data to derive business insights, and strong focus on controllership and efficiency. He or she will work closely with marketing team leadership to provide strategic guidance, as well as overall finance, operational and decision support in this exciting and evolving space. This individual will be a role model for high standards, problem solving, operational discipline, driving business insight/innovation, partnering with cr

In [1]:
t = 'Advertising & Public Relations,Aerospace Engineering,Agriculture,Animal Sciences,Anthropology & Archeology,Applied Mathematics,Architecture,Art & Music Education,History,Biological Engineering,Biomedical Engineering,Business Management & Administration,Business Economics,Chemical Engineering,Civil Engineering'
t.split(',')

['Advertising & Public Relations',
 'Aerospace Engineering',
 'Agriculture',
 'Animal Sciences',
 'Anthropology & Archeology',
 'Applied Mathematics',
 'Architecture',
 'Art & Music Education',
 'History',
 'Biological Engineering',
 'Biomedical Engineering',
 'Business Management & Administration',
 'Business Economics',
 'Chemical Engineering',
 'Civil Engineering']